# Quickstart Neo4j

Let's download and visualize the graph data about communities which is hosted on [Neo4j](https://neo4j.com/docs/). Neo4j can be querried with the [Cypher querrying language](https://neo4j.com/docs/cypher/).

## Downloading data from Neo4j 

Here is an example of using the Neo4JDownloader to retrieve all nodes from the graph. 
Adapt the Neo4JDownloader class as you see fit for retrieving information. 

In [ ]:
# Define your nodes

nodes = ["user", "repo", "org"]

# Define your relationships (edges)

relationships = {
    "member_of": {"type1": {"source": "user", "target": "org"}},
    "owner_of": {
        "type1": {"source": "user", "target": "repo"},
        "type2": {"source": "org", "target": "repo"},
    },
    "contributor_of": {
        "type1": {"source": "user", "target": "repo"},
        "type2": {"source": "org", "target": "repo"},
    },
    "parent_of": {
        "type1": {"source": "repo", "target": "repo"},
    },
}

In [ ]:
import neo4j
from utils.neo4jdownloader import Neo4JDownloader
#from dotenv import load_dotenv
from pathlib import Path
import os

#load_dotenv()  # Load environment variables from .env file

def get_downloader():
    secrets_dir = Path("/secrets")
    NEO4J_URI = (secrets_dir / "neo4j_uri").read_text()
    NEO4J_USERNAME = (secrets_dir / "neo4j_user").read_text()
    NEO4J_PASSWORD = (secrets_dir / "neo4j_password").read_text()
    NEO4J_DATABASE = (secrets_dir / "neo4j_database").read_text()

    # NEO4J_URI = os.environ.get("NEO4J_URI")
    # NEO4J_USERNAME = os.environ.get("NEO4J_USER")
    # NEO4J_PASSWORD = os.environ.get("NEO4J_PASSWORD")
    # NEO4J_DATABASE = neo4j_database

    return Neo4JDownloader(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD, NEO4J_DATABASE)

def extract_data(nodes, relationships):
    downloader = get_downloader()

    try:
        nodes_ids, nodes_features = downloader.retrieve_nodes(nodes)
        edges_indices, edges_attributes = downloader.retrieve_edges(relationships)

        return nodes_ids, nodes_features, edges_indices, edges_attributes
    finally:
        downloader.close()

In [ ]:
nodes_ids, nodes_features, edges_indices, edges_attributes = extract_data(nodes, relationships)
# example of looking at the output
# print(nodes_ids["org"])
# print(nodes_features["org"])
# print(edges_indices)

In [ ]:
from utils.builder_dataframe import neo4j_to_dataframe
df = neo4j_to_dataframe(nodes_ids, nodes_features, edges_indices, relationships)
df.head()

In [ ]:
from utils.builder_models import df_to_pydantic_models
graph = df_to_pydantic_models(df.head(200), relationships)

## Visualizing the Graph

Here is some code snippet for visualizing the graph you have downloaded from Neo4J.

**Disclaimer: if you run visualization on an entire graph of over 5000 nodes, it may take a long time or may give unreadable graphs. Be smart in your visualizations.**

For CUSTOM visualizations, if you are familiar with networkx, there is a `create_networkx_graph` function in `visualization.py` from which you can kick off other graph manipulations or visualizations.

### Visualize the entire graph or a subpart of the graph. 

We suggest: 
1. select from the dataframe (classic pandas operations)
2. convert to the pydantic models 
3. run visualizations

In [ ]:
from utils.visualization import visualize_graph
from pathlib import Path
output_path = Path("plots/graphs/graph_visualization.png")
visualize_graph(graph, output_path)

### Visualize Clusters in the graph
This function will make clusters based on the nodes that are handed over. 
We use the function from networkx `nx.weakly_connected_components(G)`

In [ ]:
from utils.visualization import visualize_clusters
from pathlib import Path
output_dir = Path("plots/clusters/")
visualize_clusters(graph, output_dir)